In [48]:
import ephem
import pandas as pd
from datetime import datetime, timedelta
import pytz


In [49]:
def create_observer(date, lat, lon):
    observer = ephem.Observer()
    observer.date = date
    observer.lat = lat
    observer.lon = lon
    observer.elevation = 0  # Set default elevation
    return observer

def get_astronomical_events(observer):
    sun = ephem.Sun()

    events = {
        'next_sunrise': observer.next_rising(sun),
        'next_sunset': observer.next_setting(sun),
        'previous_sunrise': observer.previous_rising(sun),
        'previous_sunset': observer.previous_setting(sun),
    }

    # Calculate civil twilight (dawn and dusk)
    observer.horizon = '-6'  # Civil twilight horizon

    twilight_events = {
        'next_civil_dawn': observer.next_rising(sun, use_center=True),
        'next_civil_dusk': observer.next_setting(sun, use_center=True),
        'previous_civil_dawn': observer.previous_rising(sun, use_center=True),
        'previous_civil_dusk': observer.previous_setting(sun, use_center=True),
    }

    events.update(twilight_events)

    return events

def julian_to_datetime(julian_date, timezone_obj):
    # Convert Julian date to datetime in UTC
    utc_dt = ephem.localtime(julian_date)
    # Localize to the given timezone
    return utc_dt.astimezone(timezone_obj)

def create_dataframe(date, lat, lon, timezone):
    observer = create_observer(date, lat, lon)
    events = get_astronomical_events(observer)
    
    # Convert Julian dates to datetime in the specified timezone
    timezone_obj = pytz.timezone(timezone)
    events = {event: julian_to_datetime(time, timezone_obj) for event, time in events.items()}
    
    df = pd.DataFrame(events.items(), columns=['Event', 'DateTime'])
    return df


In [50]:
# Example usage
latitude = 55.4199
longitude = 11.5428
timezone = 'Europe/Copenhagen'

now = datetime.now(pytz.utc)

df = create_dataframe(now, latitude, longitude, timezone)
print(df)

                 Event                         DateTime
0         next_sunrise 2024-07-21 15:13:11.220856+02:00
1          next_sunset 2024-07-21 20:49:35.653021+02:00
2     previous_sunrise 2024-07-20 15:15:56.827540+02:00
3      previous_sunset 2024-07-20 20:46:43.831990+02:00
4      next_civil_dawn 2024-07-21 14:05:08.185602+02:00
5      next_civil_dusk 2024-07-21 21:57:40.923588+02:00
6  previous_civil_dawn 2024-07-20 14:07:11.487474+02:00
7  previous_civil_dusk 2024-07-20 21:55:31.368008+02:00


In [51]:
import ephem
import pandas as pd
from datetime import datetime
import pytz

def create_observer(lat, lon, timezone):
    # Set the location
    observer = ephem.Observer()
    observer.lat = str(latitude)
    observer.lon = str(longitude)

    # Set the timezone
    timezone = 'Europe/Copenhagen'
    timezone_obj = pytz.timezone(timezone)
    observer.date = datetime.now(timezone_obj)

    return observer

def get_astronomical_events(observer):
    sun = ephem.Sun()
    events = {
        'next_sunrise': observer.next_rising(sun).datetime().replace(tzinfo=pytz.UTC).astimezone(timezone_obj),
        'next_sunset': observer.next_setting(sun).datetime().replace(tzinfo=pytz.UTC).astimezone(timezone_obj),
        'previous_sunrise': observer.previous_rising(sun).datetime().replace(tzinfo=pytz.UTC).astimezone(timezone_obj),
        'previous_sunset': observer.previous_setting(sun).datetime().replace(tzinfo=pytz.UTC).astimezone(timezone_obj),
    }

    # Calculate civil twilight (dawn and dusk)
    observer.horizon = '-6'  # Civil twilight horizon

    twilight_events = {
        'next_civil_dawn': observer.next_rising(sun, use_center=True).datetime().replace(tzinfo=pytz.UTC).astimezone(timezone_obj),
        'next_civil_dusk': observer.next_setting(sun, use_center=True).datetime().replace(tzinfo=pytz.UTC).astimezone(timezone_obj),
        'previous_civil_dawn': observer.previous_rising(sun, use_center=True).datetime().replace(tzinfo=pytz.UTC).astimezone(timezone_obj),
        'previous_civil_dusk': observer.previous_setting(sun, use_center=True).datetime().replace(tzinfo=pytz.UTC).astimezone(timezone_obj),
    }

    events.update(twilight_events)

    return events

def create_dataframe(lat, lon, timezone_str):
    timezone_obj = pytz.timezone(timezone_str)
    observer = create_observer(lat, lon, timezone_obj)
    events = get_astronomical_events(observer)
    
    df = pd.DataFrame(events.items(), columns=['Event', 'DateTime']).set_index('Event')
    df['simple_datetime']=df['DateTime'].dt.strftime('%H:%M %d-%m-%Y')
    return df

# Example usage
latitude = 55.4199
longitude = 11.5428
timezone = 'Europe/Copenhagen'

df = create_dataframe(latitude, longitude, timezone)
print(df)


                                            DateTime   simple_datetime
Event                                                                 
next_sunrise        2024-07-22 05:03:28.660949+02:00  05:03 22-07-2024
next_sunset         2024-07-21 21:37:38.874402+02:00  21:37 21-07-2024
previous_sunrise    2024-07-21 05:01:50.681545+02:00  05:01 21-07-2024
previous_sunset     2024-07-20 21:39:11.471837+02:00  21:39 20-07-2024
next_civil_dawn     2024-07-22 04:11:13.723053+02:00  04:11 22-07-2024
next_civil_dusk     2024-07-21 22:29:52.953817+02:00  22:29 21-07-2024
previous_civil_dawn 2024-07-21 04:09:09.987382+02:00  04:09 21-07-2024
previous_civil_dusk 2024-07-20 22:31:51.351773+02:00  22:31 20-07-2024


In [63]:
def calculate_door_time(calculate_by="daily", sunrise_offset=0, dusk_offset=0, fixed_open_hour=5, fixed_open_minute=0, fixed_close_hour=23, fixed_close_minute=0):
    if calculate_by == "daily":
        todays_info = create_dataframe(lat=latitude, lon=longitude, timezone_str=timezone)
        door_open = todays_info.loc['next_sunrise','DateTime']- timedelta(minutes=sunrise_offset)
        door_close = todays_info.loc['next_civil_dusk','DateTime']+timedelta(minutes=dusk_offset)
    elif calculate_by == "fixed":
        today = datetime.today()
        # Create a new datetime object with today's date and the fixed hour
        door_open = datetime(today.year, today.month, today.day, fixed_open_hour, fixed_open_minute)
        door_close = datetime(today.year, today.month, today.day, fixed_close_hour, fixed_close_minute)
        
    return door_open, door_close 



In [64]:
calculate_door_time(calculate_by="fixed", fixed_open_hour=6, fixed_close_hour=22)

('2024-07-21T06', '2024-07-21T22')

In [65]:
df.loc['next_sunrise','DateTime']

Timestamp('2024-07-22 05:03:28.660949+0200', tz='Europe/Copenhagen')